# Investigate required values

Investigate which values are needed to predict in order to fully specify a protein correctly

In [ ]:
import os, sys
import functools
import itertools
import multiprocessing
import warnings
import importlib
import tempfile
from pathlib import Path
import json

from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch

SRC_DIR = os.path.join(os.path.dirname(os.getcwd()), "foldingdiff")
assert os.path.isdir(SRC_DIR)
sys.path.append(SRC_DIR)
import datasets
import angles_and_coords as ac
import nerf
import tmalign  # So we can compare structural similarity
import plotting

datasets.LOCAL_DATA_DIR

In [ ]:
importlib.reload(datasets)

train_dset = datasets.CathCanonicalAnglesOnlyDataset(
    split='train',
    zero_center=True,
    pad=128,
    trim_strategy='randomcrop',
)
len(train_dset.filenames)

In [ ]:
train_dset_noised = datasets.NoisedAnglesDataset(
    train_dset,
    timesteps=1000,
)
str(train_dset_noised)

In [ ]:
importlib.reload(ac)
importlib.reload(nerf)

# https://arxiv.org/pdf/2205.04676.pdf
# N:CA:C = tau

# Full spec should be angles_to_use=["N:CA:C", "CA:C:1N", "C:1N:1CA", "phi", "psi", "omega"], dists_to_use=["N:CA", "CA:C", "C:1N"]

def test_consistency(fname:str, angles_to_use=["phi", "psi", "omega", "tau", "CA:C:1N", "C:1N:1CA"], dists_to_use=["0C:1N", "N:CA", "CA:C"], visualize:bool=False):
    """Test the consistency of reconstructing a pdb file"""
    # Create the internal coordinates
    angles = ac.canonical_distances_and_dihedrals(fname, distances=dists_to_use, angles=angles_to_use)
    if angles is None:
        return np.nan, None
    with tempfile.TemporaryDirectory() as dirname:
        out_fname = os.path.join(dirname, "rebuilt_" + os.path.basename(fname))
        # rebuilt = ac.create_new_chain(
        #     out_fname, angles,
        #     angles_to_set=angles_to_use, distances_to_set=dists_to_use
        # )
        out_fname_written = ac.create_new_chain_nerf(out_fname, angles, angles_to_set=angles_to_use, dists_to_set=dists_to_use)
        if not out_fname_written:  # Failed on the way, should have returned empty string ""
            return np.nan, None
        score = tmalign.run_tmalign(fname, out_fname)
        # angles_new = ac.canonical_distances_and_dihedrals(out_fname, distances=dists_to_use, angles=angles_to_use)
        view = None
        if visualize:
            view = view_pdb(out_fname)
    return score, view

score, view = test_consistency(train_dset.filenames[40], visualize=False)
print(score)

Generate a folding visual example

In [ ]:
from curses import savetty


importlib.reload(ac)

def view_pdb(*args, **kwargs) -> None:
    return None

def visualize_training_example(i: int = 0, timestep: int = 0, struct_pdb: str = ""):
    """Visualize the training example"""
    # Keys ['angles', 'attn_mask', 'position_ids', 'corrupted', 't', 'known_noise']
    item = train_dset_noised.__getitem__(i, use_t_val=timestep, ignore_zero_center=True)
    assert item['t'].item() == timestep
    attn_idx = torch.where(item['attn_mask'])[0]

    angles = item['corrupted'][attn_idx].cpu().numpy()
    angles_df = pd.DataFrame(angles, columns=train_dset_noised.feature_names['angles'])
    if not struct_pdb:
        with tempfile.TemporaryDirectory() as tempdir:
            fname = ac.create_new_chain_nerf(os.path.join(tempdir, "temp.pdb"), angles_df)
            return view_pdb(fname), angles
    else:
        fname = ac.create_new_chain_nerf(struct_pdb, angles_df)
        return view_pdb(fname), angles

noised_view, noised_angles = visualize_training_example(
    i=56, timestep=999,
    struct_pdb="../plots/pdb_structures/noising_visualization/fully_noised.pdb",
)
noised_view

In [ ]:
noised_angles.shape

In [ ]:
def show_matrix(mat, fname:str=""):
    """Visualize the matrix"""
    fig, ax = plt.subplots(dpi=300)
    ax.imshow(mat.T, aspect=3.5)
    ax.set(
        xticklabels=[],
        xticks=[],
        yticklabels=[],
        yticks=[],
    )
    if fname:
        fig.savefig(fname, bbox_inches='tight')
    return fig

show_matrix(
    noised_angles,
    "../plots/pdb_structures/noising_visualization/fully_noised_values.pdf"
).show()

In [ ]:
clean_view, clean_angles = visualize_training_example(
    i=56, timestep=0,
    struct_pdb="../plots/pdb_structures/noising_visualization/clean.pdb",
)
clean_view

In [ ]:
show_matrix(
    clean_angles,
    "../plots/pdb_structures/noising_visualization/clean_values.pdf"
).show()

## Look at reconstruction within training examples

In [ ]:
# Look at the full defined set, or 4 angles we currently fit, or 3 dihedrals

angle_sets_to_evaluate = [
    ["phi", "psi", "omega", "tau", "CA:C:1N", "C:1N:1CA"],
    ["phi", "psi", "omega", "tau"], 
    ["phi", "psi", "omega"],
]

# Look at either all the distances or none of them
dist_sets_to_evaluate = [
    ["0C:1N", "N:CA", "CA:C"],
    []
]

# Combinatorially look at these angle sets
def evaluate_angle_set_parallel(filenames, angles, dists):
    warnings.filterwarnings('ignore', '.*elements were guessed from atom_name.*')
    warnings.filterwarnings('ignore', '.*invalid value encountered in true_div.*')
    pfunc = functools.partial(
        test_consistency,
        angles_to_use=angles,
        dists_to_use=dists,
    )
    pool = multiprocessing.Pool(multiprocessing.cpu_count())
    tm_scores = np.array([score for score, _view in pool.map(pfunc, filenames, chunksize=20)])
    pool.close()
    pool.join()
    return tm_scores

human_readable_angle_combos = [
    r"dihedrals, angles, and distances",
    r"dihedrals and angles",
    r"dihedrals, $\theta_1$, and distances",
    r"dihedrals, $\theta_1$",
    r"dihedrals, distances",
    r"dihedrals only"
]
per_angle_dist_results = {}
for human_name, (a, d) in zip(human_readable_angle_combos, itertools.product(angle_sets_to_evaluate, dist_sets_to_evaluate)):
    print(human_name, a, d)
    per_angle_dist_results[human_name] = evaluate_angle_set_parallel(train_dset.filenames[:5000], angles=a, dists=d)

In [ ]:
# Plot these results
reconst_tmscore_dir = plotting.PLOT_DIR / "reconstruction_angles_coords"
if not reconst_tmscore_dir.is_dir():
    os.makedirs(reconst_tmscore_dir)

fig, ax = plt.subplots(dpi=300)
for i, (k, v) in enumerate(per_angle_dist_results.items()):
    c = [
        'tab:blue',
        'tab:orange',
        'tab:green',
        'tab:red',
        'tab:purple',
        'tab:brown',
    ][i]
    mean = np.nanmean(v)
    std = np.nanstd(v)
    sns.histplot(v, bins=40, stat='proportion', ax=ax, label=f"{k} - ${mean:.4f} \pm {std:.4f}$", alpha=0.5, color=c)
ax.axvline(0.5, color='grey', alpha=0.3, linestyle='--')
ax.legend(prop={'size': 6})
ax.set(
    xlabel="Reconstruction TM-score",
)
ax.set_title("Reconstruction across different angle/distance sets, training set subset", fontsize=10)
fig.savefig(reconst_tmscore_dir / "reconstruction_distributions.pdf", bbox_inches='tight')
fig.show()

In [ ]:
reconstruction_scores = pd.DataFrame(
    per_angle_dist_results
)
reconstruction_scores

In [ ]:
np.nanmean(reconstruction_scores, axis=0), np.nanstd(reconstruction_scores, axis=0)

In [ ]:
fig, ax = plt.subplots(dpi=300)
sns.boxplot(
    data=reconstruction_scores,
    whis=(5, 95),
    color=sns.color_palette()[0],
    fliersize=2,
    flierprops={"marker": "o", "alpha": 0.3, "color": "gray"},
)
ax.set_xticks(list(range(6)))
ax.set_xticklabels(reconstruction_scores.columns, rotation=45, ha='right', size=8)
ax.axhline(0.5, linestyle='--', color='grey', alpha=0.5)
ax.set(
    ylabel="TM-score of reconstruction",
    title="Reconstruction accuracy, combinations of angles/distances",
)
fig.savefig(reconst_tmscore_dir / "reconstruction_barplot.pdf", bbox_inches='tight')
fig.show()

In [ ]:
train_dset_seq_lens = np.array([train_dset[i]['lengths'].item() for i in range(5000)])
train_dset_seq_lens

In [ ]:
fig, ax = plt.subplots(dpi=300)
sns.lineplot(
    x=train_dset_seq_lens,
    y=reconstruction_scores['dihedrals and angles'],
    ax=ax,
)
ax.set(
    xlabel="Structure length",
    ylabel="TM-score of reconstruction",
    title="Structure reconstruction with dihedrals and angles",
)
fig.savefig(
    reconst_tmscore_dir / "reconstruction_by_len.pdf", bbox_inches='tight'
)
fig.show()

In [ ]:
train_dset_seq_lens

In [ ]:
nonnan_idx = np.where(~np.isnan(reconstruction_scores['dihedrals and angles'].values))
stats.spearmanr(train_dset_seq_lens[nonnan_idx], reconstruction_scores['dihedrals and angles'].values[nonnan_idx])